In [1]:
#Import libraries for doing image analysis
from skimage.io import imread
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier as RF
import glob
import os
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold as KFold
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from matplotlib import colors
from pylab import cm
from skimage import segmentation
from skimage.morphology import watershed
from skimage import measure
from skimage import morphology
import numpy as np
import pandas as pd
from scipy import ndimage
from skimage.feature import peak_local_max
import tensorflow as tf
from math import ceil, floor
import time

# make graphics inline
%matplotlib inline

/home/davide/miniconda3/envs/honours/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import warnings
warnings.filterwarnings("ignore")

#### Importing the Data

In [3]:
# get the classnames from the directory structure
directory_names = list(set(glob.glob(os.path.join("competition_data","train", "*"))\
 ).difference(set(glob.glob(os.path.join("competition_data","train","*.*")))))

#### Preparing Training Data


In [4]:
# Rescale the images and create the combined metrics and training labels

#get the total training images
numberofImages = 0
for folder in directory_names:
    for fileNameDir in os.walk(folder):   
        for fileName in fileNameDir[2]:
             # Only read in the images
            if fileName[-4:] != ".jpg":
              continue
            numberofImages += 1
            
# We'll rescale the images to be 25x25
maxPixel = 28
imageSize = maxPixel * maxPixel
num_rows = numberofImages # one row for each image in the training dataset
num_features = imageSize # + 1 # for our ratio

# X is the feature vector with one row of features per image
# consisting of the pixel values and our metric
X = np.zeros((num_rows, num_features), dtype=float)
# y is the numeric class label 
y = np.zeros((num_rows))

files = []
# Generate training data
i = 0    
label = 0
# List of string of class names
namesClasses = list()

print("Reading images")
# Navigate through the list of directories
for folder in directory_names:
    # Append the string class name for each class
    currentClass = folder.split(os.pathsep)[-1]
    namesClasses.append(currentClass)
    for fileNameDir in os.walk(folder):   
        for fileName in fileNameDir[2]:
            # Only read in the images
            if fileName[-4:] != ".jpg":
              continue

            # Read in the images and create the features
            nameFileImage = "{0}{1}{2}".format(fileNameDir[0], os.sep, fileName)            
            image = imread(nameFileImage, as_grey=True)
            files.append(nameFileImage)
            #axisratio = getMinorMajorRatio(image)
            image = resize(image, (maxPixel, maxPixel))

            # Store the rescaled image pixels and the axis ratio
            X[i, 0:imageSize] = np.reshape(image, (1, imageSize))
            #X[i, imageSize] = axisratio

            # Store the classlabel
            y[i] = label
            i += 1
            # report progress for each 5% done  
            report = [int((j+1)*num_rows/10.) for j in range(10)]
            if i in report: 
                print (np.ceil(i *100.0 / num_rows), "% done")
    label += 1

Reading images
10.0 % done
20.0 % done
30.0 % done
40.0 % done
50.0 % done
60.0 % done
70.0 % done
80.0 % done
90.0 % done
100.0 % done


## Augmenting Dataset

In [5]:
IMAGE_SIZE = maxPixel

In [6]:
def flip_images(X_imgs):
    X_flip = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (IMAGE_SIZE, IMAGE_SIZE, 1))
    tf_img1 = tf.image.flip_left_right(X)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in X_imgs:
            flipped_imgs = sess.run([tf_img1], feed_dict = {X: img})
            X_flip.extend(flipped_imgs)
    X_flip = np.array(X_flip, dtype = np.float32)
    return np.concatenate((X_imgs, X_flip))

In [7]:
def rotate_images(X_imgs):
    X_rotate = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (IMAGE_SIZE, IMAGE_SIZE, 1))
    k = tf.placeholder(tf.int32)
    tf_img = tf.image.rot90(X, k = k)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in X_imgs:
            for i in range(3):  # Rotation at 90, 180 and 270 degrees
                rotated_img = sess.run(tf_img, feed_dict = {X: img, k: i + 1})
                X_rotate.append(rotated_img)
        
    X_rotate = np.array(X_rotate, dtype = np.float32)
    return np.concatenate((X_imgs, X_rotate))

In [8]:
def scale_images(X_imgs, scales=[0.90]):
    # Various settings needed for Tensorflow operation
    boxes = np.zeros((len(scales), 4), dtype = np.float32)
    for index, scale in enumerate(scales):
        x1 = y1 = 0.5 - 0.5 * scale # To scale centrally
        x2 = y2 = 0.5 + 0.5 * scale
        boxes[index] = np.array([y1, x1, y2, x2], dtype = np.float32)
    box_ind = np.zeros((len(scales)), dtype = np.int32)
    crop_size = np.array([maxPixel, maxPixel], dtype = np.int32)
    
    X_scale_data = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (1, maxPixel, maxPixel, 1))
    # Define Tensorflow operation for all scales but only one base image at a time
    tf_img = tf.image.crop_and_resize(X, boxes, box_ind, crop_size)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for img_data in X_imgs:
            batch_img = np.expand_dims(img_data, axis = 0)
            scaled_imgs = sess.run(tf_img, feed_dict = {X: batch_img})
            X_scale_data.extend(scaled_imgs)
    
    X_scale_data = np.array(X_scale_data, dtype = np.float32)
    return np.concatenate((X_imgs, X_scale_data))

In [9]:
def get_translate_parameters(index):
    if index == 0: # Translate left 20 percent
        offset = np.array([0.0, 0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = 0
        w_end = int(ceil(0.8 * IMAGE_SIZE))
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 1: # Translate right 20 percent
        offset = np.array([0.0, -0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 2: # Translate top 20 percent
        offset = np.array([0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = int(ceil(0.8 * IMAGE_SIZE)) 
    else: # Translate bottom 20 percent
        offset = np.array([-0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        h_end = IMAGE_SIZE 
        
    return offset, size, w_start, w_end, h_start, h_end

def translate_images(X_imgs):
    offsets = np.zeros((len(X_imgs), 2), dtype = np.float32)
    n_translations = 2
    X_translated_arr = []
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(n_translations):
            X_translated = np.zeros((len(X_imgs), IMAGE_SIZE, IMAGE_SIZE, 1), 
				    dtype = np.float32)
            X_translated.fill(1.0) # Filling background color
            base_offset, size, w_start, w_end, h_start, h_end = get_translate_parameters(i)
            offsets[:, :] = base_offset 
            glimpses = tf.image.extract_glimpse(X_imgs, size, offsets)
            
            glimpses = sess.run(glimpses)
            X_translated[:, h_start: h_start + size[0], \
			 w_start: w_start + size[1], :] = glimpses
            X_translated_arr.extend(X_translated)
    X_translated_arr = np.array(X_translated_arr, dtype = np.float32)
    return np.concatenate((X_imgs, X_translated_arr))

In [10]:
def augment_dataset(X_imgs, y_imgs):
    assert len(X_imgs) == len(y_imgs)
    X_aug = []
    y_aug = []
    i = 0
    print("Starting Dataset Augmentation...")
    
    for i in range(len(y_imgs)):
        
        imgs = [np.reshape(X_imgs[i,:],(IMAGE_SIZE,IMAGE_SIZE,1))]
        label = [y_imgs[i]]
        
        imgs = flip_images(imgs)
        imgs = rotate_images(imgs)
        imgs = scale_images(imgs)
        imgs = translate_images(imgs)
        
        labels = label * len(imgs)
#         print(len(labels), imgs.shape)
        
        X_aug += [np.array(x) for x in imgs.tolist()[:]]
        y_aug += labels
        
        i += 1
        # report progress for each 5% done  
        report = [int((j+1)*len(y_imgs)/10.) for j in range(10)]
        if i in report: 
            print (np.ceil(i *100.0 / len(y_imgs)), "% done")
        
    return X_aug, y_aug

## CNN for image classification

In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [12]:
N_CLASSES = len(set(y))
print(N_CLASSES)

121


In [13]:
def cnn_model_fn(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # Plankton images are 25x25 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=121) # N_CLASSES

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [14]:
#Generate indexes to shuffle dataset and create testset

idx = np.arange(len(y))
np.random.seed(seed=1234)
np.random.shuffle(idx)
separator = ceil(len(y)/10*8)
print("Train / Total: ", separator, "/", len(y))

train_data = X[idx[:separator]].astype('float32')
eval_data = X[idx[separator:]].astype('float32')
train_labels = y[idx[:separator]].astype(int)
eval_labels = y[idx[separator:]].astype(int)

train_data = train_data #[:, 0:-1] # Returns np.array, remove axis rateo feature
eval_data = eval_data #[:, 0:-1]

print("Shapes:", train_data.shape, train_labels.shape, eval_data.shape, eval_labels.shape)

Train / Total:  24269 / 30336
Shapes: (24269, 784) (24269,) (6067, 784) (6067,)


In [21]:
def run_experiment(train_data, train_labels, eval_data, eval_labels):
    # Create the Estimator
    now = str(time.time())[-5:]
    image_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/models/convnet_model_whole_"+now)

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
    image_classifier.train(
      input_fn=train_input_fn,
      steps=50000)#,
      #hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = image_classifier.evaluate(input_fn=eval_input_fn)
#     print(eval_results)
    return eval_results

In [22]:
basic_results = run_experiment(train_data, train_labels, eval_data, eval_labels)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/models/convnet_model_whole_90707', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf88f68dd8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/models/convnet_model_whole_90707/model.ckpt.
INFO:tensorflow:loss = 4.80543, step = 1
INFO:tensorflow:global_step/sec: 49.0536
INFO:tensorflow:loss = 4.75575, step = 101 (2.023 sec)
INFO:tensorflow:global_step/sec: 27.2918
INFO:tensorflow:loss = 4.59243, step = 201 (3.653 sec)
INFO:tensorflow:global_step

INFO:tensorflow:global_step/sec: 52.1001
INFO:tensorflow:loss = 3.87146, step = 7801 (1.919 sec)
INFO:tensorflow:global_step/sec: 52.1192
INFO:tensorflow:loss = 3.66194, step = 7901 (1.919 sec)
INFO:tensorflow:global_step/sec: 52.1205
INFO:tensorflow:loss = 3.84561, step = 8001 (1.919 sec)
INFO:tensorflow:global_step/sec: 52.173
INFO:tensorflow:loss = 3.60913, step = 8101 (1.916 sec)
INFO:tensorflow:global_step/sec: 52.0855
INFO:tensorflow:loss = 3.76458, step = 8201 (1.920 sec)
INFO:tensorflow:global_step/sec: 52.3094
INFO:tensorflow:loss = 3.79157, step = 8301 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.12
INFO:tensorflow:loss = 4.02494, step = 8401 (1.919 sec)
INFO:tensorflow:global_step/sec: 52.2748
INFO:tensorflow:loss = 3.76979, step = 8501 (1.913 sec)
INFO:tensorflow:global_step/sec: 45.0984
INFO:tensorflow:loss = 3.5249, step = 8601 (2.218 sec)
INFO:tensorflow:global_step/sec: 52.191
INFO:tensorflow:loss = 3.67936, step = 8701 (1.916 sec)
INFO:tensorflow:global_step/sec: 52

INFO:tensorflow:loss = 3.35342, step = 16201 (1.916 sec)
INFO:tensorflow:global_step/sec: 51.7732
INFO:tensorflow:loss = 2.98408, step = 16301 (1.932 sec)
INFO:tensorflow:global_step/sec: 52.1939
INFO:tensorflow:loss = 3.27209, step = 16401 (1.915 sec)
INFO:tensorflow:global_step/sec: 51.8442
INFO:tensorflow:loss = 3.36789, step = 16501 (1.929 sec)
INFO:tensorflow:global_step/sec: 48.3955
INFO:tensorflow:loss = 3.35837, step = 16601 (2.067 sec)
INFO:tensorflow:global_step/sec: 46.4262
INFO:tensorflow:loss = 3.08056, step = 16701 (2.153 sec)
INFO:tensorflow:global_step/sec: 46.4169
INFO:tensorflow:loss = 3.02456, step = 16801 (2.154 sec)
INFO:tensorflow:global_step/sec: 49.3274
INFO:tensorflow:loss = 3.41056, step = 16901 (2.027 sec)
INFO:tensorflow:global_step/sec: 51.9573
INFO:tensorflow:loss = 2.93766, step = 17001 (1.925 sec)
INFO:tensorflow:global_step/sec: 51.0559
INFO:tensorflow:loss = 3.11908, step = 17101 (1.958 sec)
INFO:tensorflow:global_step/sec: 49.7322
INFO:tensorflow:loss

INFO:tensorflow:loss = 2.78484, step = 24601 (1.917 sec)
INFO:tensorflow:global_step/sec: 52.1138
INFO:tensorflow:loss = 2.68621, step = 24701 (1.919 sec)
INFO:tensorflow:global_step/sec: 52.2698
INFO:tensorflow:loss = 2.44839, step = 24801 (1.913 sec)
INFO:tensorflow:global_step/sec: 52.3252
INFO:tensorflow:loss = 2.73123, step = 24901 (1.912 sec)
INFO:tensorflow:global_step/sec: 52.0531
INFO:tensorflow:loss = 2.68673, step = 25001 (1.921 sec)
INFO:tensorflow:global_step/sec: 52.1474
INFO:tensorflow:loss = 2.81407, step = 25101 (1.918 sec)
INFO:tensorflow:global_step/sec: 52.2794
INFO:tensorflow:loss = 2.78744, step = 25201 (1.913 sec)
INFO:tensorflow:global_step/sec: 52.2785
INFO:tensorflow:loss = 2.52062, step = 25301 (1.913 sec)
INFO:tensorflow:global_step/sec: 52.2666
INFO:tensorflow:loss = 2.42786, step = 25401 (1.913 sec)
INFO:tensorflow:global_step/sec: 52.2753
INFO:tensorflow:loss = 3.14173, step = 25501 (1.913 sec)
INFO:tensorflow:global_step/sec: 52.2291
INFO:tensorflow:loss

INFO:tensorflow:loss = 2.43275, step = 32901 (1.918 sec)
INFO:tensorflow:global_step/sec: 52.215
INFO:tensorflow:loss = 2.71481, step = 33001 (1.915 sec)
INFO:tensorflow:global_step/sec: 52.449
INFO:tensorflow:loss = 2.49507, step = 33101 (1.907 sec)
INFO:tensorflow:global_step/sec: 52.3268
INFO:tensorflow:loss = 2.24472, step = 33201 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.3074
INFO:tensorflow:loss = 2.78486, step = 33301 (1.912 sec)
INFO:tensorflow:global_step/sec: 52.1263
INFO:tensorflow:loss = 2.04893, step = 33401 (1.918 sec)
INFO:tensorflow:global_step/sec: 52.4178
INFO:tensorflow:loss = 2.67749, step = 33501 (1.908 sec)
INFO:tensorflow:global_step/sec: 52.3235
INFO:tensorflow:loss = 2.53653, step = 33601 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.3303
INFO:tensorflow:loss = 2.37756, step = 33701 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.1548
INFO:tensorflow:loss = 2.40742, step = 33801 (1.917 sec)
INFO:tensorflow:global_step/sec: 52.4382
INFO:tensorflow:loss =

INFO:tensorflow:loss = 2.35248, step = 41301 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.1531
INFO:tensorflow:loss = 2.42716, step = 41401 (1.917 sec)
INFO:tensorflow:global_step/sec: 51.4597
INFO:tensorflow:loss = 2.15004, step = 41501 (1.944 sec)
INFO:tensorflow:global_step/sec: 52.2192
INFO:tensorflow:loss = 2.27264, step = 41601 (1.915 sec)
INFO:tensorflow:global_step/sec: 52.2277
INFO:tensorflow:loss = 1.87999, step = 41701 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.325
INFO:tensorflow:loss = 2.19955, step = 41801 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.3137
INFO:tensorflow:loss = 2.24622, step = 41901 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.3099
INFO:tensorflow:loss = 2.23569, step = 42001 (1.912 sec)
INFO:tensorflow:global_step/sec: 52.1967
INFO:tensorflow:loss = 2.20567, step = 42101 (1.916 sec)
INFO:tensorflow:global_step/sec: 52.2537
INFO:tensorflow:loss = 2.03914, step = 42201 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.304
INFO:tensorflow:loss =

INFO:tensorflow:loss = 2.11361, step = 49701 (1.917 sec)
INFO:tensorflow:global_step/sec: 52.1262
INFO:tensorflow:loss = 2.0058, step = 49801 (1.918 sec)
INFO:tensorflow:global_step/sec: 52.2453
INFO:tensorflow:loss = 2.39467, step = 49901 (1.914 sec)
INFO:tensorflow:Saving checkpoints for 50000 into /tmp/models/convnet_model_whole_90707/model.ckpt.
INFO:tensorflow:Loss for final step: 2.09739.
INFO:tensorflow:Starting evaluation at 2018-02-21-12:23:40
INFO:tensorflow:Restoring parameters from /tmp/models/convnet_model_whole_90707/model.ckpt-50000
INFO:tensorflow:Finished evaluation at 2018-02-21-12:23:41
INFO:tensorflow:Saving dict for global step 50000: accuracy = 0.454755, global_step = 50000, loss = 2.09395


In [17]:
train_data_AUGM, train_labels_AUGM = augment_dataset(train_data, train_labels)

train_data_AUGM = np.array(train_data_AUGM, dtype='float32')
train_labels_AUGM = np.array(train_labels_AUGM, dtype='int')

print("Augmented dataset: ", train_data_AUGM.shape)

Starting Dataset Augmentation...
10.0 % done
20.0 % done
30.0 % done
40.0 % done
50.0 % done
60.0 % done
70.0 % done
80.0 % done
90.0 % done
100.0 % done
Augmented dataset:  (1164912, 28, 28, 1)


In [26]:
print("Starting time: ",time.time())
augmented_results = run_experiment(train_data_AUGM, train_labels_AUGM, eval_data, eval_labels)
print("Finish time: ",time.time())

Starting time:  1519216168.5230048
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/models/convnet_model_whole_18677', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc0ce8fd3c8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/models/convnet_model_whole_18677/model.ckpt.
INFO:tensorflow:loss = 4.82166, step = 1
INFO:tensorflow:global_step/sec: 50.4516
INFO:tensorflow:loss = 4.72989, step = 101 (1.981 sec)
INFO:tensorflow:global_step/sec: 52.0436
INFO:tensorflow:loss = 4.72633, step = 201 (1.9

INFO:tensorflow:loss = 3.76249, step = 7701 (1.924 sec)
INFO:tensorflow:global_step/sec: 52.2375
INFO:tensorflow:loss = 4.08461, step = 7801 (1.915 sec)
INFO:tensorflow:global_step/sec: 52.1219
INFO:tensorflow:loss = 4.18978, step = 7901 (1.918 sec)
INFO:tensorflow:global_step/sec: 52.0845
INFO:tensorflow:loss = 3.66745, step = 8001 (1.920 sec)
INFO:tensorflow:global_step/sec: 52.1092
INFO:tensorflow:loss = 3.92775, step = 8101 (1.919 sec)
INFO:tensorflow:global_step/sec: 52.1217
INFO:tensorflow:loss = 3.86999, step = 8201 (1.918 sec)
INFO:tensorflow:global_step/sec: 52.0648
INFO:tensorflow:loss = 3.98896, step = 8301 (1.921 sec)
INFO:tensorflow:global_step/sec: 52.032
INFO:tensorflow:loss = 4.13955, step = 8401 (1.922 sec)
INFO:tensorflow:global_step/sec: 49.148
INFO:tensorflow:loss = 3.92249, step = 8501 (2.034 sec)
INFO:tensorflow:global_step/sec: 51.7391
INFO:tensorflow:loss = 3.99102, step = 8601 (1.933 sec)
INFO:tensorflow:global_step/sec: 52.1386
INFO:tensorflow:loss = 4.43482, 

INFO:tensorflow:global_step/sec: 51.99
INFO:tensorflow:loss = 3.75991, step = 16201 (1.923 sec)
INFO:tensorflow:global_step/sec: 52.2574
INFO:tensorflow:loss = 3.48906, step = 16301 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.0437
INFO:tensorflow:loss = 3.11996, step = 16401 (1.921 sec)
INFO:tensorflow:global_step/sec: 52.2476
INFO:tensorflow:loss = 3.46282, step = 16501 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.2382
INFO:tensorflow:loss = 3.04516, step = 16601 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.2844
INFO:tensorflow:loss = 3.42817, step = 16701 (1.913 sec)
INFO:tensorflow:global_step/sec: 52.1987
INFO:tensorflow:loss = 3.49313, step = 16801 (1.916 sec)
INFO:tensorflow:global_step/sec: 52.2582
INFO:tensorflow:loss = 3.90321, step = 16901 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.2271
INFO:tensorflow:loss = 3.27861, step = 17001 (1.915 sec)
INFO:tensorflow:global_step/sec: 52.1743
INFO:tensorflow:loss = 3.45204, step = 17101 (1.917 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 52.3489
INFO:tensorflow:loss = 3.31885, step = 24601 (1.910 sec)
INFO:tensorflow:global_step/sec: 52.2564
INFO:tensorflow:loss = 3.57272, step = 24701 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.1837
INFO:tensorflow:loss = 3.14727, step = 24801 (1.916 sec)
INFO:tensorflow:global_step/sec: 52.234
INFO:tensorflow:loss = 2.80089, step = 24901 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.0843
INFO:tensorflow:loss = 2.90972, step = 25001 (1.920 sec)
INFO:tensorflow:global_step/sec: 52.2502
INFO:tensorflow:loss = 3.30019, step = 25101 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.3028
INFO:tensorflow:loss = 2.94297, step = 25201 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.1828
INFO:tensorflow:loss = 3.97047, step = 25301 (1.917 sec)
INFO:tensorflow:global_step/sec: 52.1481
INFO:tensorflow:loss = 4.11691, step = 25401 (1.918 sec)
INFO:tensorflow:global_step/sec: 52.2487
INFO:tensorflow:loss = 3.20998, step = 25501 (1.913 sec)
INFO:tensorflow:globa

INFO:tensorflow:global_step/sec: 52.0911
INFO:tensorflow:loss = 3.02311, step = 32901 (1.919 sec)
INFO:tensorflow:global_step/sec: 52.1043
INFO:tensorflow:loss = 3.07065, step = 33001 (1.919 sec)
INFO:tensorflow:global_step/sec: 52.1687
INFO:tensorflow:loss = 3.13543, step = 33101 (1.917 sec)
INFO:tensorflow:global_step/sec: 52.1889
INFO:tensorflow:loss = 3.06587, step = 33201 (1.916 sec)
INFO:tensorflow:global_step/sec: 52.1778
INFO:tensorflow:loss = 2.83146, step = 33301 (1.917 sec)
INFO:tensorflow:global_step/sec: 52.0604
INFO:tensorflow:loss = 2.60405, step = 33401 (1.921 sec)
INFO:tensorflow:global_step/sec: 52.0669
INFO:tensorflow:loss = 3.14856, step = 33501 (1.921 sec)
INFO:tensorflow:global_step/sec: 52.0967
INFO:tensorflow:loss = 2.98755, step = 33601 (1.919 sec)
INFO:tensorflow:global_step/sec: 52.4456
INFO:tensorflow:loss = 3.09642, step = 33701 (1.907 sec)
INFO:tensorflow:global_step/sec: 52.18
INFO:tensorflow:loss = 3.00149, step = 33801 (1.917 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 52.2222
INFO:tensorflow:loss = 2.66343, step = 41301 (1.915 sec)
INFO:tensorflow:global_step/sec: 52.062
INFO:tensorflow:loss = 3.09373, step = 41401 (1.921 sec)
INFO:tensorflow:global_step/sec: 52.0535
INFO:tensorflow:loss = 2.89088, step = 41501 (1.921 sec)
INFO:tensorflow:global_step/sec: 52.1278
INFO:tensorflow:loss = 3.61151, step = 41601 (1.918 sec)
INFO:tensorflow:global_step/sec: 52.3012
INFO:tensorflow:loss = 2.88864, step = 41701 (1.912 sec)
INFO:tensorflow:global_step/sec: 52.2166
INFO:tensorflow:loss = 2.77893, step = 41801 (1.915 sec)
INFO:tensorflow:global_step/sec: 52.3156
INFO:tensorflow:loss = 3.2482, step = 41901 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.2642
INFO:tensorflow:loss = 2.67839, step = 42001 (1.913 sec)
INFO:tensorflow:global_step/sec: 52.1263
INFO:tensorflow:loss = 2.68958, step = 42101 (1.918 sec)
INFO:tensorflow:global_step/sec: 52.2332
INFO:tensorflow:loss = 2.8055, step = 42201 (1.915 sec)
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 52.2005
INFO:tensorflow:loss = 2.44671, step = 49701 (1.916 sec)
INFO:tensorflow:global_step/sec: 52.3002
INFO:tensorflow:loss = 2.6981, step = 49801 (1.911 sec)
INFO:tensorflow:global_step/sec: 52.1573
INFO:tensorflow:loss = 3.20521, step = 49901 (1.917 sec)
INFO:tensorflow:Saving checkpoints for 50000 into /tmp/models/convnet_model_whole_18677/model.ckpt.
INFO:tensorflow:Loss for final step: 2.7243.
INFO:tensorflow:Starting evaluation at 2018-02-21-12:45:38
INFO:tensorflow:Restoring parameters from /tmp/models/convnet_model_whole_18677/model.ckpt-50000
INFO:tensorflow:Finished evaluation at 2018-02-21-12:45:39
INFO:tensorflow:Saving dict for global step 50000: accuracy = 0.237185, global_step = 50000, loss = 3.09843
Finish time:  1519217141.1529636


In [25]:
print("Training with 10000 steps")
print("BASIC RESULTS:  {'accuracy': 0.20224163, 'loss': 3.4980001, 'global_step': 10000}")
print("AUGMENTED RESULTS:  {'accuracy': 0.11719136, 'loss': 3.9900112, 'global_step': 10000}")

Training with 10000 steps
BASIC RESULTS:  {'accuracy': 0.20224163, 'loss': 3.4980001, 'global_step': 10000}
AUGMENTED RESULTS:  {'accuracy': 0.11719136, 'loss': 3.9900112, 'global_step': 10000}


In [24]:
print("Training with 50000 steps")
print("BASIC RESULTS: ", basic_results)

Training with 50000 steps
BASIC RESULTS:  {'accuracy': 0.45475525, 'loss': 2.0939481, 'global_step': 50000}


In [27]:
print("AUGMENTED RESULTS: ", augmented_results)

AUGMENTED RESULTS:  {'accuracy': 0.23718476, 'loss': 3.0984316, 'global_step': 50000}
